In [1]:
import pandas as pd
import numpy as np
import re
import os

# Variables history

In [2]:
url2kwikly = 'https://documentation-snds.health-data-hub.fr/files/Cnam/2019-05-09_Cnam_KWIKLY-Katalogue-Sniiram-SNDS-v1.3_MPL-2.0.xlsm'

raw_excel = pd.ExcelFile(url2kwikly)
table_names = raw_excel.sheet_names
undesired_tables = [
    'Accueil', 'DCIR', 'PMSI', 'CAUSES DE DECES',
    'id_potentiel_médicales_DCIR',
    'id_potentiel_médicales_PMSI',
    'id_potentiel_médicales_CDD',
    'IR_BEN_R', 'IR_IMB_R', 'IR_PHA_R'
    ]
table_names = [t for t in table_names if t not in undesired_tables]
tables = {tab_name: pd.read_excel(raw_excel, sheet_name=tab_name, header=3) for tab_name in table_names}

In [3]:
def create_histrory(x, years):
    var_history = []
    for c in years:
        if x[c] == 'X' and c != '*':
            var_history.append(int(c))
    if '*' in years and x['*'] == 'X':
        var_history += [int(y) for y in np.arange(2013, 2020)]
    var_history.sort()
    return var_history

In [5]:
variables_history = {}
for table, df in tables.items():
    # exception management
    if 'Nom variable' not in df.columns:
        df.rename(columns={
            'Nom de la variable': 'Nom variable', 
            'Variable': 'Nom variable', 
            'Nom Variable': 'Nom variable',
            'Code variable': 'Nom variable',
            'Nomvariable': 'Nom variable'}, inplace=True)
    #print(table)
    variables_history[table] = pd.DataFrame()
    years = [c for c in df.columns if re.search('^20', str(c)) is not None]
    if '*' in df.columns:
        years.append('*')
    variables_history[table]['history'] = df.apply(lambda x: create_histrory(x, years), axis=1)
    variables_history[table]['var'] = df['Nom variable']
    variables_history[table]['table'] = table
    variables_history[table] = variables_history[table].loc[:, ['table', 'var', 'history']]

    
all_variables_history_df = pd.concat(variables_history.values(), axis=0)
# filter out residual garbage lines from excel
all_variables_history_df = all_variables_history_df.loc[~(all_variables_history_df['history'].apply(len) == 0), :]

In [7]:
def compute_start(x):
    return min(x)

def compute_end(x):
    return max(x)

def compute_missings(x):
    missings = [str(y) for y in np.arange(min(x), max(x) + 1) if y not in x]
    if len(missings) > 0:
        return '_'.join(missings)
    else:
        return np.NAN

In [8]:
all_variables_history_df['created'] = all_variables_history_df['history'].apply(compute_start)
all_variables_history_df['deleted'] = all_variables_history_df['history'].apply(compute_end)
all_variables_history_df['missings'] = all_variables_history_df['history'].apply(compute_missings)
all_variables_history_df.head()

,table,var,history,created,deleted,missings
0,ER_ARO_F,ARO_ASU_NAT,"[2006, 2007, 2008, 2009, 2010, 2011, 2012, 201...",2006,2019,NaN
1,ER_ARO_F,ARO_CPL_COD,"[2006, 2007, 2008, 2009, 2010, 2011, 2012, 201...",2006,2019,NaN
2,ER_ARO_F,ARO_ENV_TYP,"[2006, 2007, 2008, 2009, 2010, 2011, 2012, 201...",2006,2019,NaN
3,ER_ARO_F,ARO_FJH_TYP,"[2006, 2007, 2008, 2009, 2010, 2011, 2012, 201...",2006,2019,NaN
4,ER_ARO_F,ARO_FTA_COD,"[2006, 2007, 2008, 2009, 2010, 2011, 2012, 201...",2006,2019,NaN


In [9]:
print('Missing years for:')
missings_years = all_variables_history_df.loc[~all_variables_history_df['missings'].isna(), :]
print(len(missings_years))
missings_years

Missing years for:
27


,table,var,history,created,deleted,missings
53,T_HADXXB,DGN_PAL,"[2006, 2007, 2008, 2009, 2010, 2011, 2014, 201...",2006,2017,2012_2013
64,T_HADXXB,ETA_NUM_GEO,"[2006, 2017]",2006,2017,2007_2008_2009_2010_2011_2012_2013_2014_2015_2016
39,T_HADXXC,ETA_NUM_GEO,"[2006, 2017]",2006,2017,2007_2008_2009_2010_2011_2012_2013_2014_2015_2016
0,T_HADXXGRP,ETA_NUM_GEO,"[2006, 2017]",2006,2017,2007_2008_2009_2010_2011_2012_2013_2014_2015_2016
11,T_HADXXGRE,ETA_NUM_GEO,"[2006, 2017]",2006,2017,2007_2008_2009_2010_2011_2012_2013_2014_2015_2016
0,T_HADXXFP,TYP_ART,"[2014, 2017]",2014,2017,2015_2016
1,T_HADXXFP,ETA_NUM_EPMSI,"[2014, 2017]",2014,2017,2015_2016
4,T_HADXXFP,RHAD_NUM,"[2014, 2017]",2014,2017,2015_2016
5,T_HADXXFP,FAC_NUM,"[2014, 2017]",2014,2017,2015_2016
6,T_HADXXFP,ENT_MOI,"[2014, 2017]",2014,2017,2015_2016


## Saving parsed variables history

In [11]:
all_variables_history_df.drop('history', axis=1).to_csv('variables_history.csv', header=True, index=False)

# Tables history

In [23]:
raw_excel = pd.ExcelFile(url2kwikly)
table_names = raw_excel.sheet_names
desired_tables = ['PMSI', 'CAUSES DE DECES']
table_names = [t for t in table_names if t in desired_tables]
raw_tables_history = {tab_name: pd.read_excel(raw_excel, sheet_name=tab_name, header=7) for tab_name in table_names}

In [24]:
tables_history = {}
for table, df in raw_tables_history.items():
    #print(table)
    tables_history[table] = pd.DataFrame()
    years = [c for c in df.columns if re.search('^20', str(c)) is not None]
    tables_history[table]['var'] = df['Table']
    tables_history[table]['history'] = df.apply(lambda x: create_histrory(x, years), axis=1)

all_tables_history_df = pd.concat(tables_history.values(), axis=0)
# filter out residual garbage lines from excel
all_tables_history_df = all_tables_history_df.loc[~(all_tables_history_df['history'].apply(len) == 0), :]

In [25]:
all_tables_history_df['created'] = all_tables_history_df['history'].apply(compute_start)
all_tables_history_df['deleted'] = all_tables_history_df['history'].apply(compute_end)
all_tables_history_df['missings'] = all_tables_history_df['history'].apply(compute_missings)
all_tables_history_df.head()

,var,history,created,deleted,missings
0,T_MCOXXA,"[2006, 2007, 2008, 2009, 2010, 2011, 2012, 201...",2006,2017,NaN
1,T_MCOXXB,"[2006, 2007, 2008, 2009, 2010, 2011, 2012, 201...",2006,2017,NaN
2,T_MCOXXC,"[2006, 2007, 2008, 2009, 2010, 2011, 2012, 201...",2006,2017,NaN
3,T_MCOXXCSTC,"[2007, 2008, 2009, 2010, 2011, 2012, 2013, 201...",2007,2017,NaN
4,T_MCOXXD,"[2006, 2007, 2008, 2009, 2010, 2011, 2012, 201...",2006,2017,NaN


In [26]:
print('Missing years for:')
missings_years = all_tables_history_df.loc[~all_tables_history_df['missings'].isna(), :]
print(len(missings_years))
missings_years

Missing years for:
1


,var,history,created,deleted,missings
87,T_HADXXEHPA,"[2011, 2012, 2015, 2016, 2017]",2011,2017,2013_2014


## Saving parsed tables history

In [27]:
all_tables_history_df.drop('history', axis=1).to_csv('tables_history.csv', header=True, index=False)